In [48]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import scipy.stats as stats
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional

In [22]:
import scipy
from scipy.stats import norm

In [23]:
stock='AXISBANK.NS'
data = yf.download(stock,'2005-01-01','2023-05-05')
data1=data

[*********************100%***********************]  1 of 1 completed


In [24]:
close_prices = data['Adj Close']

min_value = close_prices.min()
max_value = close_prices.max()
normalized_data = (close_prices - min_value) / (max_value - min_value)
normalized_data

Date
2005-01-03    0.002702
2005-01-04    0.002610
2005-01-05    0.001597
2005-01-06    0.001764
2005-01-07    0.002684
                ...   
2023-04-27    0.913188
2023-04-28    0.890224
2023-05-02    0.901652
2023-05-03    0.890063
2023-05-04    0.896609
Name: Adj Close, Length: 4525, dtype: float64

In [25]:
columns_to_normalize = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
min_values = data[columns_to_normalize].min()
max_values = data[columns_to_normalize].max()
normalized_data = (data[columns_to_normalize] - min_values) / (max_values - min_values)
print(normalized_data)
data=normalized_data

                Open      High       Low     Close  Adj Close    Volume
Date                                                                   
2005-01-03  0.001788  0.001671  0.003257  0.003139   0.002702  0.011789
2005-01-04  0.003101  0.001886  0.003692  0.003031   0.002610  0.009257
2005-01-05  0.002854  0.001446  0.001640  0.001855   0.001597  0.009649
2005-01-06  0.001594  0.001457  0.001520  0.002049   0.001764  0.009894
2005-01-07  0.000851  0.001864  0.002313  0.003117   0.002684  0.012695
...              ...       ...       ...       ...        ...       ...
2023-04-27  0.918162  0.914288  0.915852  0.912745   0.913188  0.132671
2023-04-28  0.915039  0.914342  0.889957  0.889664   0.890224  0.190839
2023-05-02  0.898779  0.899717  0.901466  0.901151   0.901652  0.107415
2023-05-03  0.892156  0.889485  0.890391  0.889502   0.890063  0.125293
2023-05-04  0.888011  0.889807  0.894137  0.896082   0.896609  0.080938

[4525 rows x 6 columns]


In [26]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2005-01-03,0.001788,0.001671,0.003257,0.003139,0.002702,0.011789
2005-01-04,0.003101,0.001886,0.003692,0.003031,0.002610,0.009257
2005-01-05,0.002854,0.001446,0.001640,0.001855,0.001597,0.009649
2005-01-06,0.001594,0.001457,0.001520,0.002049,0.001764,0.009894
2005-01-07,0.000851,0.001864,0.002313,0.003117,0.002684,0.012695
...,...,...,...,...,...,...
2023-04-27,0.918162,0.914288,0.915852,0.912745,0.913188,0.132671
2023-04-28,0.915039,0.914342,0.889957,0.889664,0.890224,0.190839
2023-05-02,0.898779,0.899717,0.901466,0.901151,0.901652,0.107415


In [27]:
returns = np.diff(data['Adj Close'])
copyreturns=np.diff(data1['Adj Close'])
copyreturns=-copyreturns
returns=-returns

In [28]:
data.shape,returns.shape

((4525, 6), (4524,))

In [29]:
returns.shape

(4524,)

In [30]:
np.sum(returns)

-0.8939064173861064

In [31]:
data['Adj Close']

Date
2005-01-03    0.002702
2005-01-04    0.002610
2005-01-05    0.001597
2005-01-06    0.001764
2005-01-07    0.002684
                ...   
2023-04-27    0.913188
2023-04-28    0.890224
2023-05-02    0.901652
2023-05-03    0.890063
2023-05-04    0.896609
Name: Adj Close, Length: 4525, dtype: float64

In [32]:
mean = np.mean(returns)
std_dev = np.std(returns)
mean,std_dev

(-0.00019759204628340106, 0.009668438095499244)

In [33]:
np.array(returns)

array([ 9.28709763e-05,  1.01224483e-03, -1.67156272e-04, ...,
       -1.14283354e-02,  1.15892609e-02, -6.54576923e-03])

In [34]:
np.sort(np.array(returns))

array([-0.08373783, -0.07999809, -0.06792979, ...,  0.06450123,
        0.08153718,  0.12803812])

In [35]:
import torch
import torch.nn as nn
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

In [36]:
def quantile_loss(q, y, y_p):
        e = y-y_p
        return tf.keras.backend.mean(tf.keras.backend.maximum(q*e, (q-1)*e))


In [37]:
input=data['Adj Close'][0:-1]
input.shape,returns.shape

((4524,), (4524,))

In [58]:
quantile = 0.95
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(100,1)))
model.add(Dropout(0.1)) 
model.add(LSTM(units=50))
model.add(Dense(1))
model.compile(loss=lambda y, y_p: quantile_loss(quantile, y, y_p), optimizer='adam')



In [59]:
model.fit(input,returns,epochs=100)

Epoch 1/100
142/142 [==============================] - 3s 2ms/step - loss: 0.0010
Epoch 2/100
142/142 [==============================] - 0s 2ms/step - loss: 9.8935e-04
Epoch 3/100
142/142 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 4/100
142/142 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 5/100
142/142 [==============================] - 0s 3ms/step - loss: 9.9219e-04
Epoch 6/100
142/142 [==============================] - 0s 2ms/step - loss: 9.7696e-04
Epoch 7/100
142/142 [==============================] - 0s 3ms/step - loss: 9.9608e-04
Epoch 8/100
142/142 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 9/100
142/142 [==============================] - 0s 2ms/step - loss: 9.8301e-04
Epoch 10/100
142/142 [==============================] - 0s 2ms/step - loss: 9.9679e-04
Epoch 11/100
142/142 [==============================] - 0s 2ms/step - loss: 9.8353e-04
Epoch 12/100
142/142 [==============================] - 0s 3ms/step 

142/142 [==============================] - 0s 2ms/step - loss: 9.7001e-04
Epoch 96/100
142/142 [==============================] - 0s 2ms/step - loss: 9.6552e-04
Epoch 97/100
142/142 [==============================] - 0s 2ms/step - loss: 9.7424e-04
Epoch 98/100
142/142 [==============================] - 0s 2ms/step - loss: 9.7929e-04
Epoch 99/100
142/142 [==============================] - 0s 2ms/step - loss: 9.6888e-04
Epoch 100/100
142/142 [==============================] - 0s 2ms/step - loss: 9.6822e-04


In [63]:
pred=model.predict(input[4071:])
loss=quantile_loss(0.95,returns[4071:],pred)
loss

15/15 [==============================] - 0s 1ms/step


<tf.Tensor: shape=(), dtype=float64, numpy=0.0015609379515205506>

In [64]:
pred_denormalized = (pred * (max_values['Close'] - min_values['Close'])) + min_values['Close']
returns_denormalized = (returns * (max_values['Close'] - min_values['Close'])) + min_values['Close']
loss=quantile_loss(0.96,returns_denormalized[4071:],pred_denormalized)
loss

<tf.Tensor: shape=(), dtype=float64, numpy=1.2406025484279928>